**What is database and SQL**

1. Data Storage in Databases:
- Most data we work with is stored in databases.
- Databases are systems designed to store and manage data efficiently.
- They are like organized digital filing cabinets where data is kept in a structured way.

2. Relational Databases:

- It organizes data into tables, which can be linked—or related—based on data common to each. 
- The most common type of database is a relational database.
- Examples include PostgreSQL, MySQL, and SQL Server.
- These databases store data in tables (similar to spreadsheets), and we can search or query the data using a language called SQL (Structured Query Language).

3. SQL:
- SQL is a key tool for data scientists.
- It’s a language used to access and manipulate data in databases.
- With SQL, we can retrieve data, insert new data, update existing data, and delete data we no longer need.

4. NotQuiteABase:
- This is a Python-based tool that mimics some functions of a real database but is simpler to help us learn.
- NotQuiteABase is a teaching tool.
- By working through examples and problems with this tool, we will get a sense of how databases work and how SQL is used to solve real-world data problems.

# 1. CREATE TABLE and INSERT

- <u>Database</u> is a collection of tables, where we organize our data. And table is collection of rows like a matrix.
- Table is also associated with a fixed <u>Schema</u>.
- <u>Schema</u> is conceptual framework which indicates the column entries of the table and their data type. e.g. columns like 'user_id', 'name', 'num_friends' in a table, say 'users'.

- Examples: Let's say we have a users data -

- SQL is almost completely case and indentation insensitive
- The capitalization and indentation style is a preferred style here.
- If you start learning SQL, you will surely encounter other examples styled differently.

## NotQUuiteABase (Python implementation of DB and SQL)

- Here, we will create a 'Table' by specifying above schema. To insert rows we will use insert methods.
- Behind the scene - Each row will be stored in a dict from column names to values.
- We will do so by creating NotQuiteABase project using python to learn about DB and SQL concepts.

**Handling Null values in SQL databases**

- In a typical SQL database, you explicitly specify whether a column can contain null (None) values. For simplicity, we will use any column to have null values.

**Implementing Dunder Methods** 
- The implementation introduces dunder methods to treat a table as a List[Row]. This approach is particularly useful for testing the code.

In [1]:
# Import and define type aliases

from typing import Tuple, Sequence, List, Any, Callable, Dict, Iterator
from collections import defaultdict

# type aliases we will use later
Row = Dict[str, Any]   # A database row is stored as a dict form like {name:'Alice', 'age': 30, ...}
WhereClause = Callable[[Row], bool]  # Predicate for a single row
HavingClause = Callable[[List[Row]], bool] # Predicate over multiple rows

In [15]:
# Defining table class

class Table:
    def __init__(self, columns: List[str], types: List[type]) -> None:
        assert len(columns) == len(types)  # Check if types for all columns are given

        self.columns = columns      # Names of columns
        self.types = types         # column data types
        self.rows: List[Row] = []   # rows are empty now and remember Row in rows will be in dict form

# Helper method to get type of a column
    def col2type(self, col: str) -> type:
        idx = self.columns.index(col)    # Find index of the 'col' column in columns list
        return self.types[idx]           # return type of it.

# 1. INSERT
    
# Define insert method that checks if inserted values are valid

    def insert(self, values: list) -> None:
        # Check if # of values are same as # columns /types
        if (len(values)) != len(self.types):
            raise ValueError(f"You need to provide {len(self.types)} values")
        # Check for right types of values
        for value, typ3 in zip(values, self.types):
            if not isinstance(value, typ3) and value is not None:  # check if value is instance of type typ3 and its nonempty
                raise TypeError(f"Expected type {typ3} but got {value}")
        self.rows.append(dict(zip(self.columns, values)))

# Define Dunder methods to allow us to treat a table like a List[Row]
    def __getitem__(self, idx: int) -> Row:    # Allows to use [] to access item of iterable (list/dict etc)
        return self.rows[idx]

    def __iter__(self) -> Iterator[Row]:       # Allows us using loop by making object iterable
        return iter(self.row)
   
    def __len__(self) -> int:                  # Returns object's length so that we can use len function on class instances
        return len(self.rows)

# Method to pretty print our table
    def __repr__(self):                        # Returns string representaion of object
        rows = "\n".join(str(row) for row in self.rows)
        return f"{self.columns}\n{rows}"

# 2. UPDATE

# Let's add UPDATE method in our custom class 
    def update(self, 
               updates: Dict[str, Any],                     # {column name : value}
               predicate: WhereClause = lambda row: True):  # Until predicate function not defined explicitely it 
                                                            # will change all row values of given column
                                                            # Predicate is basically WHERE 
        # First check the updates have valid name and type
        for column, new_value in updates.items():
            if column not in self.columns:
                raise ValueError("Invalid column: {column}")
    
            typ3 = self.col2type(column)
            if not isinstance(new_value, typ3) and new_value is not None:
                raise TypeError(f"exapected type {typ3}, but got {new_value}")
                
            # Now update
        for row in self.rows:
            if predicate(row):
                for column, new_value in updates.items():
                    row[column] = new_value


# 3. DELETE
    def delete(self, predicate: WhereClause = lambda row: True)  -> None:  # By default predicate is True for all rows
        """
        Delete all rows matching predicate
        """
        self.rows = [row for row in self.rows if not predicate(row)]

# 4. SELECT

    def select(self, keep_columns: List[str] = None, additional_columns: Dict[str, Callable] = None) -> 'Table':
        # Quoting 'Table' in type annotation allows using the class name as a type hint, 
        # even if it's not yet defined or to avoid circular dependencies.

        if keep_columns is None:             # If no columns specified return all columns
            keep_columns = self.columns

        if additional_columns is None:
            additional_columns = {}

        # New columns names
        new_columns = keep_columns + list(additional_columns.keys())  

        # keep_columns types
        keep_types = [self.col2type(col) for col in keep_columns]

        add_types = [calculation.__annotations__['return'] 
                     for calculation in additional_columns.values()]  # annotation method stores the types,
                                                                      # based on already defined annotations
        # Create a new table for results
        new_table = Table(new_columns, keep_types+add_types)

        for row in self.rows:
            new_row = [row[column] for column in keep_columns]
            for column_name, calculation in additional_columns.items():
                new_row.append(calculation(row))
            new_table.insert(new_row)

        return new_table

# Where and limit methods

    def where(self, predicate: WhereClause = lambda row: True) -> 'Table':
        """
        Return the rows that satisfy the predicate supplied
        """
        where_table = Table(self.columns, self.types)
        for row in self.rows:
            if predicate(row):
                values = [row[column] for column in self.columns]
                where_table.insert(values)
        return where_table
        
    def limit(self, num_rows: int) -> 'Table':
        """ 
        Return only the first 'num_rows' rows
        """
        limit_table = Table(self.columns, self.types)
        for i, row in enumerate(self.rows):
            if i >= num_rows:
                break
            values = [row[column] for column in self.columns]
            limit_table.insert(values)
        return limit_table

# 5. GROUP BY


    def group_by(self,
                 group_by_columns: List[str],
                 aggregates: Dict[str, Callable],
                 having: HavingClause = lambda group: True) -> 'Table':
        
        grouped_rows = defaultdict(list)
        
        # Populate groups
        for row in self.rows:
            key = tuple(row[column] for column in group_by_columns)  # because dict key cannot be list, take it as tuple
            
            grouped_rows[key].append(row)
            
        # Result table consists of group_by columns and aggregates
        new_columns = group_by_columns + list(aggregates.keys())
        group_by_types = [self.col2type(col) for col in group_by_columns]
        aggregate_types = [agg.__annotations__['return']
                           for agg in aggregates.values()]
        result_table = Table(new_columns, group_by_types + aggregate_types)

        for key, rows in grouped_rows.items():
            if having(rows):
                new_row = list(key)
                for aggregate_name, aggregate_fn in aggregates.items():
                    new_row.append(aggregate_fn(rows))
                result_table.insert(new_row)
        return result_table

# 6. ORDER BY

    def order_by(self, order: Callable[[Row], Any]) -> 'Table':
        new_table = self.select()  # make a copy
        new_table.rows.sort(key=order)
        return new_table

# 7. JOIN

    def join(self, other_table: 'Table', left_join: bool = False) -> 'Table':
        join_on_columns = [c for c in self.columns  # Columns in both tables
                           if c in other_table.columns] 
        additional_columns = [c for c in other_table.columns # Columns only in right table
                              if c not in join_on_columns] 
        
        # all columns from left table + additional_columns from right table
        new_columns = self.columns + additional_columns
        new_types = self.types + [other_table.col2type(col) 
                                  for col in additional_columns]
        join_table = Table(new_columns, new_types)
        
        for row in self.rows:
            def is_join(other_row):
                return all(other_row[c] == row[c] for c in join_on_columns)
            
            other_rows = other_table.where(is_join).rows
            
            # Each other row that matches this one produces a result row.
            for other_row in other_rows:
                join_table.insert([row[c] for c in self.columns] +
                                  [other_row[c] for c in additional_columns])
            
            # If no rows match and it's a left join, output with Nones.
            if left_join and not other_rows:
                join_table.insert([row[c] for c in self.columns] +
                                  [None for c in additional_columns])
        return join_table
    
       

In [3]:
# Let's create a instance of 'Table' class

users = Table(['user_id', 'name', 'num_friends'], [int, str, int])

users.insert([0, "Hero", 0])
users.insert([1, "Dunn", 2])
users.insert([2, "Sue", 3])
users.insert([3, "Chi", 3])
users.insert([4, "Thor", 3])
users.insert([5, "Clive", 2])
users.insert([6, "Hicks", 3])
users.insert([7, "Devin", 2])
users.insert([8, "Kate", 2])
users.insert([9, "Klein", 3])
users.insert([10, "Jen", 1])

print(users)      # When we print the instance it runs dunder repr
                  # thus giving table string

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}


- List-like API (Application programming interface) makes it easy to write tests
- By implementing a list-like API, you enable users of your class to interact with its instances in familiar ways that they would interact with lists. This makes your custom data structures intuitive and easy to use.
- what is meant by List-like API here?
  > - We created a class which consists of defined dunders to characterize its instances to act like list of rows(which is a dict)
  > - And we can use python's in built list operations toa access the class instance's elements.

In [11]:
# Use attributes from class to access table specifications

# 1. Access row
users.__getitem__(idx=5)

{'user_id': 5, 'name': 'Clive', 'num_friends': 2}

In [12]:
# 2. Length of table
users.__len__()

11

In [13]:
# See we can now access the table with list features of python
assert len(users) == 11
assert users[1]['name'] == 'Dunn'

# 2. UPDATE

- In SQL when we want to update data in our table, we need to specify following:
1. What table to update
2. Which rows to update
3. Which fields to update
4. What their new values should be

- in sql the query would be:  
UPDATE users  
SET num_friends = 3  
WHERE user_id = 1;

- Let's add this update method in our custom class--
  (See Table class)

In [14]:
# Let's check update method

# assert users[1]['num_friends'] == 2 # original value

# Let's update it to 11
users.update({'num_friends':11}, lambda row: row['user_id'] == 1)
assert users[1]['num_friends'] == 11  # It was originally 2

# 3. DELETE
In sql to delete all rows we use query:

1. This will delete ALL rows  
DELETE FROM users;

2. Only delete first row  
DELETE FROM users WHERE user_id = 1;

- If you supply a predicate function (i.e., a WHERE clause), this deletes only the rows that satisfy it.
- If you don’t supply one, the default predicate always returns True, and you will delete every row.

See how we did this in our class.

In [115]:
# Let's check DELETE method

# Let's first insert a row

users.insert([11, 'Modiji', 1])
print(users)   # this added 11th user

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 11}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}
{'user_id': 11, 'name': 'Modiji', 'num_friends': 1}


In [16]:
users.delete(predicate = lambda row: row['user_id'] == 11)

In [17]:
print(users)   # this deleted 11th user

# users.delete() # this will delete all rows

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 11}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}


# 4. SELECT

- To inspect the sql table, we use SELECT statement:

1. To get the entire contents  
SELECT * FROM users;

2. To get the first two rows  
SELECT * FROM users LIMIT 2;

3. To only get specific columns  
SELECT user_id FROM users;

4. To only get specific rows  
SELECT user_id FROM users WHERE name = 'Dunn';

- We can also use SELECT statements to calculate fields:  
SELECT LENGTH(name) AS name_length FROM users;

- We will define a SELECT method in our Table class that will return a new Table.
- The method accepts two optional arguments:
1. keep_columns specifies the names of the columns you want to keep in the result.
   - If you don’t supply it, the result contains all the columns.
2. additional_columns is a dictionary whose keys are new column names and whose values are functions specifying how to compute
the values of the new columns.
    - We’ll peek at the type annotations of those functions to figure out the types of the new columns, so the functions will need to have annotated return types.
  
- If we will not supply any of these two, we will get copy of our table.

**`__annotations__`**
see : <link> http://localhost:8888/notebooks/Data-science-from-scratch/Some_basic_concepts.ipynb

- Our select returns a new Table, while the typical SQL SELECT just produces some sort of transient result set (unless you explicitly insert the results into a table).

In [118]:
# Let's check SELECT method

# sql -- SELECT * FROM users;
all_users = users.select()
all_users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 11}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}

In [119]:
# sql -- SELECT * FROM users LIMIT 2;
two_users = users.limit(2)
two_users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 11}

In [120]:
# sql -- SELECT user_id FROM users;
just_ids = users.select(keep_columns=['user_id'])
just_ids

['user_id']
{'user_id': 0}
{'user_id': 1}
{'user_id': 2}
{'user_id': 3}
{'user_id': 4}
{'user_id': 5}
{'user_id': 6}
{'user_id': 7}
{'user_id': 8}
{'user_id': 9}
{'user_id': 10}

In [121]:
# sql -- SELECT user_id FROM users WHERE name = 'Dunn';

dunn_ids = (users.where(lambda row: row['name'] == 'Dunn').select(keep_columns=['user_id']))  # for multiline fluent queries,
                                                                                              # we wrap it in paranthesis
dunn_ids

['user_id']
{'user_id': 1}

In [122]:
# sql -- SELCET LENGTH(name) AS name_length FROM users;
def name_length(row) -> int: 
        return len(row['name'])

name_lengths = users.select(keep_columns=[],
                            additional_columns={'name_length':name_length})
name_lengths

['name_length']
{'name_length': 4}
{'name_length': 4}
{'name_length': 3}
{'name_length': 3}
{'name_length': 4}
{'name_length': 5}
{'name_length': 5}
{'name_length': 5}
{'name_length': 4}
{'name_length': 5}
{'name_length': 3}

# 5. GROUP BY

- The GROUP BY clause in SQL is used to arrange identical data into groups.
- This allows you to perform aggregate functions like SUM, COUNT, MIN, MAX, and AVG on these groups.


- 'group by' will take column names based on which grouping is to be done.
- It takes aggregation function which is a dict
- It take optional 'having' predicate which is similar to 'where' but works on multiple rows

In [123]:
# Example: Name Length Metrics
# Group based on users name length
# For each group, calculate min_user_id and total number of users

def min_user_id(rows) -> int:
    return min(row["user_id"] for row in rows)

def length(rows) -> int:
    return len(rows)

stats_by_length = (
    users
    .select(additional_columns={"name_length" : name_length})
    .group_by(group_by_columns=["name_length"],
              aggregates={"min_user_id" : min_user_id,
                          "num_users" : length}))

stats_by_length

    

['name_length', 'min_user_id', 'num_users']
{'name_length': 4, 'min_user_id': 0, 'num_users': 4}
{'name_length': 3, 'min_user_id': 2, 'num_users': 3}
{'name_length': 5, 'min_user_id': 5, 'num_users': 4}

In [124]:
# Example: First Letter Metrics
# Group based on user's name's first character
# Calculate average number of friends for each group
# Only show groups whose avg firends > 1

def first_letter_of_name(row: Row) -> str:
    return row["name"][0] if row["name"] else ""

def average_num_friends(rows: List[Row]) -> float:
    return sum(row["num_friends"] for row in rows) / len(rows)

def enough_friends(rows: List[Row]) -> bool:
    return average_num_friends(rows) > 1

avg_friends_by_letter = (
    users
    .select(additional_columns={'first_letter' : first_letter_of_name})
    .group_by(group_by_columns=['first_letter'],
              aggregates={"avg_num_friends" : average_num_friends},
              having=enough_friends)
)

avg_friends_by_letter


['first_letter', 'avg_num_friends']
{'first_letter': 'H', 'avg_num_friends': 1.5}
{'first_letter': 'D', 'avg_num_friends': 6.5}
{'first_letter': 'S', 'avg_num_friends': 3.0}
{'first_letter': 'C', 'avg_num_friends': 2.5}
{'first_letter': 'T', 'avg_num_friends': 3.0}
{'first_letter': 'K', 'avg_num_friends': 2.5}

In [125]:
# Example: User ID sum
# Calculate sum of IDs of all users whose user_id > 1.

def sum_user_ids(rows: List[Row]) -> int:
    return sum(row["user_id"] for row in rows)

user_id_sum = (
    users
    .where(lambda row: row["user_id"] > 1)
    .group_by(group_by_columns=[],
              aggregates={ "user_id_sum" : sum_user_ids })
)
user_id_sum

['user_id_sum']
{'user_id_sum': 54}

1. Using 'group_by' we can group any rows based on specific columns

2. Can perform specific calculations using **Aggregate functions**

3. Real databses do this more efficiently


# 6. ORDER BY

- This clause is used to sort results
- For Example, you want to sort user names alphabatically and show top two, the sql will look like:
  
  SELECT * FROM users  
  ORDER BY name  
  LIMIT 2;


In [126]:
# Example: Ordering average friends by letter

# Find average friends of users name starting with same letter
# Order them 
# list only top 4 friendliest

friendliest_letters = (avg_friends_by_letter 
                       .order_by(lambda row: -row["avg_num_friends"])
                       .limit(4)
                      )
friendliest_letters

['first_letter', 'avg_num_friends']
{'first_letter': 'D', 'avg_num_friends': 6.5}
{'first_letter': 'S', 'avg_num_friends': 3.0}
{'first_letter': 'T', 'avg_num_friends': 3.0}
{'first_letter': 'C', 'avg_num_friends': 2.5}

- In sql, with `ORDER BY` we can also specify `ASC`(ascending) or `DESC`(descending).
- In our NotQuietBase implementation, we need to specify this in our 'order' method. (Like -row["avg_num_friends"])

# 7. JOIN

**Normalization:**

- Relational databases are often normalized to minimize redundancy. This means data is split into different tables to avoid storing the same information multiple times.

- Example: Instead of storing a list of interests for each user in one table, we create a separate table (user_interests) to store the relationship between user_ids and interests.

In [127]:
# Create a interests table

user_interests = Table(['user_id', 'interest'], [int, str])
user_interests.insert([0, "SQL"])
user_interests.insert([0, "NoSQL"])
user_interests.insert([2, "SQL"])
user_interests.insert([2, "MySQL"])

# Note on Redundancy: There's redundancy in storing the same interest multiple times. 
#In real databases, this is often handled by using an additional table for interests with unique interest_ids.


**Joining Tables**

- To analyze data that spans multiple tables, we use the JOIN operation to combine rows from two tables based on related columns.

1. INNER JOIN:
   - Combines rows from both tables where the specified condition is met.
   - Only returns rows with matching entries in both tables.

2. LEFT JOIN:
   - Returns all rows from the left table and matched rows from the right table.
   - If there's no match, the result will have NULL values for columns from the right table.
  
3. Additional JOIN Types
- RIGHT JOIN: Keeps rows from the right table that have no matches in the left table.

- FULL OUTER JOIN: Keeps rows from both tables that have no matches.

- Note: The NotQuiteABase implementation doesn't include RIGHT JOIN or FULL OUTER JOIN.

In [129]:
# Finding Users Interested in SQL

sql_users = (
    users
    .join(user_interests)
    .where(lambda row: row["interest"] == "SQL")
    .select(keep_columns=["name"])
)
sql_users

['name']
{'name': 'Hero'}
{'name': 'Sue'}

In [131]:
# Counting User Interests with LEFT JOIN

def count_interests(rows: List[Row]) -> int:
    """Counts how many rows have non-None interests"""
    return len([row for row in rows if row["interest"] is not None])

user_interest_counts = (
    users
    .join(user_interests, left_join=True)
    .group_by(group_by_columns=["user_id"],
              aggregates={"num_interests": count_interests})
)
user_interest_counts

['user_id', 'num_interests']
{'user_id': 0, 'num_interests': 2}
{'user_id': 1, 'num_interests': 0}
{'user_id': 2, 'num_interests': 2}
{'user_id': 3, 'num_interests': 0}
{'user_id': 4, 'num_interests': 0}
{'user_id': 5, 'num_interests': 0}
{'user_id': 6, 'num_interests': 0}
{'user_id': 7, 'num_interests': 0}
{'user_id': 8, 'num_interests': 0}
{'user_id': 9, 'num_interests': 0}
{'user_id': 10, 'num_interests': 0}

# 8. SUBQUERIES

# 9. INDEXES

# 10. Query OPTIMIZATION

# 11. NoSQL